<a href="https://colab.research.google.com/github/Biswajit0408/FlashCard/blob/main/backend_v_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF transformers sentence-transformers nltk pandas


In [ ]:
import fitz
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_name


Saving DIWALI THEORY QUESTION AND ANSWER.pdf to DIWALI THEORY QUESTION AND ANSWER (2).pdf


'DIWALI THEORY QUESTION AND ANSWER (2).pdf'

In [ ]:
def extract_text(file):
    if file.endswith(".pdf"):
        doc = fitz.open(file)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    else:
        return open(file, "r", encoding="utf-8", errors="ignore").read()

raw_text = extract_text(file_name)
raw_text[:1000]


'DIWALI THEORY QUESTION AND ANSWER \n \nSubmitted By: Rithvika Rout \nClass: X \n \n1. How can information technology be used for celebrating festivals like Diwali? \n \nAns: Diwali can be celebrated in a variety of entertaining and inventive ways using information \ntechnology. Through social media or WhatsApp, people can send electronic greetings, well \nwishes, and celebratory notes. Online shopping makes it simple for us to purchase clothing, \ndecorations, and gifts. We can also send our loved ones pictures, videos, and reels of our \nfestivities. No matter where they are, individuals can keep in touch and celebrate the holiday \ntogether thanks to online pujas and virtual Diwali celebrations. \n \n2. Mention any 3 mobile apps or websites that help during Diwali and explain their use. \n \nAns: Three mobile apps or websites that help during Diwali are Blinkit, Canva, and Zomato. \n●\u200b Blinkit helps people get last-minute Diwali essentials like sweets, snacks, and diyas \ndeliv

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
sentences = sent_tokenize(raw_text)
len(sentences)


32

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def top_k_sentences(text, k=15):
    sents = sent_tokenize(text)
    if len(sents) <= k:
        return sents
    vec = TfidfVectorizer().fit_transform(sents)
    scores = vec.sum(axis=1).A1
    top_idx = sorted(range(len(scores)), key=lambda x: scores[x], reverse=True)[:k]
    return [sents[i] for i in top_idx]

important_sentences = top_k_sentences(raw_text, 15)
important_sentences


['No matter where they are, individuals can keep in touch and celebrate the holiday \ntogether thanks to online pujas and virtual Diwali celebrations.',
 '●\u200b Canva can be used to design beautiful Diwali cards, posters, or social media posts to \nshare festive wishes creatively.',
 '●\u200b Blinkit helps people get last-minute Diwali essentials like sweets, snacks, and diyas \ndelivered quickly to their homes.',
 '●\u200b Zomato helps order delicious festive food and sweets from restaurants when people \nwant to celebrate without cooking.',
 '●\u200b  Enjoy virtual fireworks shows or online celebrations instead of bursting crackers to \nprevent air and noise pollution.',
 'Ans: Three mobile apps or websites that help during Diwali are Blinkit, Canva, and Zomato.',
 'Through social media or WhatsApp, people can send electronic greetings, well \nwishes, and celebratory notes.',
 '●\u200b  Use online shopping platforms to buy eco-friendly diyas, decorations, and gifts safely \nfrom ho

In [ ]:
def extract_answer_phrase(sentence):
    doc = nlp(sentence)
    noun_phrases = [chunk.text for chunk in doc.noun_chunks]

    if noun_phrases:
        return max(noun_phrases, key=len)
    return sentence.split()[0]  # fallback


In [ ]:
def format_qg_input(sentence):
    answer = extract_answer_phrase(sentence)
    highlighted = sentence.replace(answer, f"<hl> {answer} <hl>")
    return highlighted, answer


In [ ]:
model_name = "valhalla/t5-base-qg-hl"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
def generate_question(sentence):
    highlighted, answer = format_qg_input(sentence)

    input_text = f"generate question: {highlighted}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True)

    outputs = model.generate(
        inputs,
        max_length=64,
        num_beams=4,
        early_stopping=True
    )

    question = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {
        "question": question,
        "answer": answer,
        "context": sentence
    }


In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def dedupe(cards):
    questions = [c["question"] for c in cards]
    embeddings = embedder.encode(questions, convert_to_tensor=True)
    keep = []
    used = set()

    for i, emb in enumerate(embeddings):
        if i in used:
            continue
        keep.append(cards[i])
        sims = util.cos_sim(emb, embeddings)[0]
        close = (sims > 0.78).nonzero().tolist()
        for idx in close:
            used.add(idx[0])
    return keep

flashcards = dedupe(flashcards)
len(flashcards)


14

In [ ]:
df = pd.DataFrame(flashcards)
df


,question,answer
0,What is the name of the online pujas and Diwal...,"No matter where they are, individuals can keep..."
1,What can be used to create Diwali cards?,●​ Canva can be used to design beautiful Diwal...
2,What is the name of the app that Blinkit uses ...,●​ Blinkit helps people get last-minute Diwali...
3,What does Zomato do to help people celebrate w...,●​ Zomato helps order delicious festive food a...
4,What is the purpose of fireworks?,●​ Enjoy virtual fireworks shows or online ce...
5,What are the three apps that help during Diwali?,Ans: Three mobile apps or websites that help d...
6,What can people send through social media?,"Through social media or WhatsApp, people can s..."
7,What do you do to make eco-friendly gifts?,●​ Use online shopping platforms to buy eco-f...
8,What can you do to help reduce paper waste?,Ans: \n●​ Send digital greetings or e-cards i...
9,What is the main reason for Diwali?,Ans: Diwali can be celebrated in a variety of ...


In [ ]:
df.to_csv("flashcards.csv", index=False)
files.download("flashcards.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>